In [1]:
#### INSTALLATION OF JAVA/SPARK
!sudo apt update
!sudo apt install openjdk-17-jdk -y
#!curl -JLO 'https://apache.osuosl.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz'
!tar xvf spark-3.3.1-bin-hadoop3.tgz
!mv spark-3.3.1-bin-hadoop3 /opt/spark

#### INSTALLING LIBRARIES
!pip install findspark
!pip install pyspark
!pip install pytorch-ignite transformers parallelformers

Get:1 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 https://deb.nodesource.com/node_16.x focal InRelease [4583 B]            
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]        
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]      
Get:5 https://deb.nodesource.com/node_16.x focal/main amd64 Packages [774 B] 0m
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]0m
Get:7 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [177 kB]33m
Get:8 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Get:9 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB] 
Get:10 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1275 kB]    
Get:11 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]0m
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [30.2 kB]
Get:13 http://archive.ubuntu.com/ubunt

### Load Dataset

In [3]:
# !pip install gdown
# !mkdir data
# !cd data
# !gdown 1QhtF1UAbVMJyAcqVJDucYwmib0hHCP1A
# !mv 'Dataset Yelp.zip' data.zip
# !unzip data.zip
# !mv  -v /notebooks/data/Dataset\ Yelp/* /notebooks/data/
# !rm -rf /notebooks/data/Dataset\ Yelp

### Checking GPU

In [ ]:
#### CHECKING CUDA SUPPORT AND VERSION
#import torch
#torch.cuda.is_available()
#!nvcc --version
#!nvidia-smi -q
#!nvidia-smi --list-gpus

### Setting SPARK

In [3]:
#### SPARK
#### DIRECTORY SETTING FOR SPARK 
import os
os.environ["JAVA_HOME"] = "/usr"
os.environ["SPARK_HOME"] = "/opt/spark"
import findspark
findspark.init();
from pyspark.sql import SparkSession
import pyspark.pandas as ps

spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.2.3")\
    .getOrCreate();

spark.sparkContext.setLogLevel("OFF");

ps.set_option('compute.ops_on_diff_frames', True);

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e3511bd3-d46c-4ea6-b20a-055160b2009a;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in 

22/11/22 15:51:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Loading the dataset

In [148]:
reviews = ps.read_json("./data/review.json", lines=True)

/opt/spark/spark-3.3.1-bin-hadoop3/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_json`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


Functions for limiting number of words to the model

In [155]:
def get_word_count(str):
    if str is None:
        return 0
    ls = str.split()
    return len(ls)

def chop_500(str):
    ls = str.split()
    return ' '.join(ls[:380])

In [171]:
reviews['word_count'] = reviews['text'].apply(get_word_count)
reviews['text'] = reviews['text'].apply(chop_500)
reviews['word_count_check'] = reviews['text'].apply(get_word_count)
reviews.loc[reviews['word_count_check']>350,:].shape

(205086, 11)

In [167]:
reviews.describe()

,cool,funny,stars,useful,word_count,word_count_check
count,6.990280e+06,6.990280e+06,6.990280e+06,6.990280e+06,6.990280e+06,6.990280e+06
mean,4.986175e-01,3.265596e-01,3.748584e+00,1.184609e+00,1.034024e+02,1.016503e+02
std,2.172460e+00,1.688729e+00,1.478705e+00,3.253767e+00,9.034197e+01,8.375869e+01
min,-1.000000e+00,-1.000000e+00,1.000000e+00,-1.000000e+00,1.000000e+00,1.000000e+00
25%,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,4.200000e+01,4.200000e+01
50%,0.000000e+00,0.000000e+00,4.000000e+00,0.000000e+00,7.500000e+01,7.500000e+01
75%,0.000000e+00,0.000000e+00,5.000000e+00,1.000000e+00,1.330000e+02,1.330000e+02
max,4.040000e+02,7.920000e+02,5.000000e+00,1.182000e+03,4.960000e+02,3.800000e+02


In [5]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="textattack/albert-base-v2-yelp-polarity", device=0)

In [4]:
AllData = spark.read \
      .option("header", True) \
      .json("./data/review.json")

AllData = AllData.select("text", "review_id")

In [24]:
DF1, DF2, DF3, DF4, DF5, DF6, DF7, DF8, DF9, DF10 = AllData.randomSplit([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], seed=123)

In [6]:
df = AllData.limit(7000)

In [7]:
from pyspark.sql.types import StructType,StructField,FloatType,Row, StringType
from pyspark.sql import functions as F

def get_label(x):
    return Row('Out1', 'Out2')(classifier(x, truncation=True).pop(0).get('label'), classifier(x,truncation=True).pop(0).get('score'))

schema = StructType([
    StructField("Label", StringType(), False),
    StructField("Score", FloatType(), False)])

example_udf = F.UserDefinedFunction(get_label, schema)
newDF = df.withColumn("Output", example_udf(df["text"]))
newDF = newDF.select("review_id", "Output.*")

In [8]:
newDF.show(truncate=15)

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1012: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


+---------------+-------+----------+
|      review_id|  Label|     Score|
+---------------+-------+----------+
|KU_O5udG6zpx...|LABEL_0|0.98213863|
|BiTunyQ73aT9...|LABEL_1| 0.9999385|
|saUsX_uimxRl...|LABEL_1|0.99986064|
|AqPFMleE6RsU...|LABEL_1|0.99991775|
|Sx8TMOWLNuJB...|LABEL_1| 0.9764664|
|JrIxlS1TzJ-i...|LABEL_0| 0.9991316|
|6AxgBCNX_PNT...|LABEL_1|0.99992704|
|_ZeMknuYdlQc...|LABEL_1|0.99986184|
|ZKvDG2sBvHVd...|LABEL_0| 0.9952799|
|pUycOfUwM8vq...|LABEL_0| 0.9977696|
|rGQRf8UafX7O...|LABEL_1|0.99945754|
|l3Wk_mvAog6X...|LABEL_1| 0.9988385|
|XW_LfMv0fV21...|LABEL_1| 0.9995548|
|8JFGBuHMoiND...|LABEL_1|0.99915755|
|UBp0zWyH60Hm...|LABEL_1|0.99959487|
|OAhBYw8IQ6wl...|LABEL_1|0.99974436|
|oyaMhzBSwfGg...|LABEL_1|0.99964035|
|LnGZB0fjfgeV...|LABEL_1|0.99991727|
|u2vzZaOqJ2fe...|LABEL_1| 0.9991479|
|Xs8Z8lmKkosq...|LABEL_1| 0.9999299|
+---------------+-------+----------+
only showing top 20 rows



In [9]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="./data/review.json", field="text")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

JSONDecodeError: Extra data: line 2 column 1 (char 709)

In [222]:
#newDF.write.csv("/data/datacsv.csv", header=True)

In [11]:
from parallelformers import parallelize

parallelize(classifier.model, num_gpus=2, fp16=True, verbose='detail')

Process ParallelProcess-2:
Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.9/dist-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/parallelformers/parallel/process.py", line 251, in run
    engine = ParallelEngine(
  File "/usr/local/lib/python3.9/dist-packages/parallelformers/parallel/engine.py", line 53, in __init__
    self.mp_group = self.create_process_group(backend)
  File "/usr/local/lib/python3.9/dist-packages/parallelformers/parallel/engine.py", line 106, in create_process_group
    torch.cuda.set_device(int(os.getenv("LOCAL_RANK", "0")))
  File "/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py", line 314, in set_device
    torch._C._cuda_setDevice(device)
RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported 

In [ ]:
inputs = tokenizer("Parallelformers is", return_tensors="pt")

outputs = model.generate(
    **inputs,
    num_beams=5,
    no_repeat_ngram_size=4,
    max_length=15,
)

print(f"Output: {tokenizer.batch_decode(outputs)[0]}")

In [188]:
reviews_tiny = ps.DataFrame(reviews.loc[0:2, "text"])

In [10]:
classifier('I hate this place, food is awfull', truncation=True)[0]['score']


0.9997188448905945

In [176]:
def classify(x):
    label = classifier(x, truncation=True)[0]['label']
    score = classifier(x, truncation=True)[0]['score']
    return label, score

In [ ]:
import pyspark.pandas as ps

data = {"col_1": [1,2,3], "col_2": [4,5,6]}
df = ps.DataFrame(data)

median_series = df[["col_1","col_2"]].apply(lambda x: x.median(), axis=1)
median_series.name = "median"

df = ps.merge(df, median_series, left_index=True, right_index=True, how='left')

In [189]:
reviews_tiny['label'],reviews_tiny['score'] = reviews_tiny['text'].apply(classify)

PandasNotImplementedError: The method `pd.Series.__iter__()` is not implemented. If you want to collect your data as an NumPy array, use 'to_numpy()' instead.

In [175]:
reviews_tiny.loc[reviews_tiny['todo'] == None, :].shape

(0, 2)

In [144]:
# def splitter(str):
#     ls, lx = str.split()[0], str.split()[1]
#     return ls,lx

In [178]:
reviews_tiny['label'] = reviews_tiny['todo'].apply(lambda x: str(x).split(',')[0])

In [182]:
str(['LABEL_0','0.9821387529373169']).split(',')[0]

"['LABEL_0'"

In [179]:
reviews_tiny.head()

,text,todo,label
0,"If you decide to eat here, just be aware it is...","[LABEL_0, 0.9821387529373169]",['LABEL_0' '0.9821387529373169']
1,I've taken a lot of spin classes over the year...,"[LABEL_1, 0.9999384880065918]",['LABEL_1' '0.9999384880065918']
2,Family diner. Had the buffet. Eclectic assortm...,"[LABEL_1, 0.9998606443405151]",['LABEL_1' '0.9998606443405151']
3,"Wow! Yummy, different, delicious. Our favorite...","[LABEL_1, 0.99991774559021]",['LABEL_1' '0.99991774559021']
4,Cute interior and owner (?) gave us tour of up...,"[LABEL_1, 0.9764665365219116]",['LABEL_1' '0.9764665365219116']
